In [3]:
import tablestore as ts

In [5]:
endpoint = 'https://kool.cn-hangzhou.ots.aliyuncs.com'
accessid = 'LTAIhnpvdDOvc1nE'
accesskey = 'Wk4cWkrWhZ3hRuJHmA0LJj63BJHY18'
instance='kool'
client = ts.OTSClient(endpoint, accessid, accesskey, instance)

In [7]:
try:
    list_response = client.list_table()
    print('table list:')
    table_names = []
    for table_name in list_response:
        table_names.append(table_name)
        print(table_name)
except Exception as e:
    print(f'list table failed, {repr(e)}')

table list:
alarm_def
alarm_messages
almdef
area
base_volt
bay
calc_expressions
calc_operands
calc_point
channel
company
data_type
data_type_value
dev_subtype
dev_type
device
interval_info
interval_type
master_info
measure_type
obj_value
point
point_index
point_rev_index
point_value
protocol
route
route_stat
rtu
rtu_stat
season
source
station
system_config
system_version
tag_info
tag_type
tap_ctrl
task_config
test_data
test_data_result
volt_level
zero_band


In [12]:
try:
    for table_name in table_names:
        describe_response  = client.describe_table(table_name)
        print('TableName: %s' % describe_response.table_meta.table_name)
        print('PrimaryKey: %s' % describe_response.table_meta.schema_of_primary_key)
        print(f'Columns: {describe_response.table_meta.column}')
        break
except Exception as e:
    print(f'list table failed, {repr(e)}')

TableName: alarm_def
PrimaryKey: [('id', 'STRING')]
list table failed, AttributeError("'TableMeta' object has no attribute 'column'")


In [24]:
def make_pk_inf_values(pk):
    inclusive_start_primary_key = [(item[0], ts.INF_MIN) for item in pk]
    exclusive_end_primary_key = [(item[0], ts.INF_MAX) for item in pk]
    return inclusive_start_primary_key, exclusive_end_primary_key

def query(client, table_name, pk):
    inclusive_start_primary_key, exclusive_end_primary_key = make_pk_inf_values(pk)
    columns_to_get = []
    limit = 10
    try:
        _, _, row_list, _ = client.get_range(
            table_name, ts.Direction.FORWARD,
            inclusive_start_primary_key, exclusive_end_primary_key,
            columns_to_get, limit
        )
        for row in row_list:
            print(row.primary_key, row.attribute_columns)
        return row_list
    except ts.OTSClientError as e:
        print("get row failed, http_status:%d, error_message:%s" % (e.get_http_status(), e.get_error_message()))
    except ts.OTSServiceError as e:
        print("get row failed, http_status:%d, error_code:%s, error_message:%s, request_id:%s" % (e.get_http_status(), e.get_error_code(), e.get_error_message(), e.get_request_id()))
    return None

In [31]:
with open('kemp_ots.txt', 'wt', encoding='utf-8') as f:
    try:
        for table_name in table_names:
            describe_response  = client.describe_table(table_name)
            print('TableName: %s\n' % describe_response.table_meta.table_name)
            print('PrimaryKey: %s\n' % describe_response.table_meta.schema_of_primary_key)
            f.write('TableName: %s\n' % describe_response.table_meta.table_name)
            f.write('PrimaryKey: %s\n' % describe_response.table_meta.schema_of_primary_key)
            row_list = query(client, table_name, describe_response.table_meta.schema_of_primary_key)
            f.write('values:\n')
            for row in row_list:
                row_line = [(column[0], column[1]) for column in row.attribute_columns]
                f.write(str(row.primary_key))
                f.write('\n')
                f.write(str(row_line))
                f.write('\n\n')
            f.write('------\n')
    except Exception as e:
        print(f'list table failed, {repr(e)}')

TableName: alarm_def

PrimaryKey: [('id', 'STRING')]

[('id', 'ACCIDENT_ALM')] [('alias', '', 1558795942583), ('alm_color', 1, 1558795942583), ('alm_mode', 4, 1558795942583), ('alm_priority', 3, 1558795942583), ('alm_type', 'ALM_ACK', 1558795942583), ('description', '事故跳闸', 1558795942583), ('messages', '{"ALM": {"message_formatter": "{alm_time} \\u4e8b\\u6545\\u8df3\\u95f8 {station_desc}  {point_desc}  {value_desc}"}}', 1558795942583), ('name', 'ACCIDENT_ALM', 1558795942583)]
[('id', 'ACTDEACT')] [('alias', '', 1558795942583), ('alm_color', 1, 1558795942583), ('alm_mode', 4, 1558795942583), ('alm_priority', 3, 1558795942583), ('alm_type', 'INH_ENA', 1558795942583), ('description', '禁止刷新', 1558795942583), ('messages', '{"CLEAN": {"message_formatter": "{alm_time}  \\u6fc0\\u6d3b\\u5237\\u65b0 {station_desc} {obj_desc} {comment} \\u8282\\u70b9\\uff1a{node} \\u64cd\\u4f5c\\u5458\\uff1a{user_name}"}, "SET": {"message_formatter": "{alm_time}  \\u7981\\u6b62\\u5237\\u65b0 {station_desc} {obj_

[('id', '0.22kV')] [('alias', '0.22kV', 1562230453084), ('description', '0.22kV', 1562230453084), ('name', '0.22kV', 1562230453084), ('ref_volt_value', 0.22, 1562230453084), ('volt_hlimit', 0.24, 1562230453084), ('volt_llimit', 0.22, 1562230453084)]
[('id', '0.4kV')] [('alias', '0.4kV', 1562230453084), ('description', '0.4kV', 1562230453084), ('name', '0.4kV', 1562230453084), ('ref_volt_value', 0.4, 1562230453084), ('volt_hlimit', 0.42, 1562230453084), ('volt_llimit', 0.38, 1562230453084)]
[('id', '1000kV')] [('alias', '1000kV', 1562230453084), ('description', '1000kV', 1562230453084), ('name', '1000kV', 1562230453084), ('ref_volt_value', 1050.0, 1562230453084), ('volt_hlimit', 1100.0, 1562230453084), ('volt_llimit', 900.0, 1562230453084)]
[('id', '10kV')] [('alias', '10kV', 1562230453084), ('description', '10kV', 1562230453084), ('name', '10kV', 1562230453084), ('ref_volt_value', 10.5, 1562230453084), ('volt_hlimit', 11.0, 1562230453084), ('volt_llimit', 9.0, 1562230453084)]
[('id', '

[('id', 'ACCUMULATE')] [('alias', '', 1558794497092), ('catalog', 'E', 1558794497092), ('description', '电度量', 1558794497092), ('name', 'ACCUMULATE', 1558794497092)]
[('id', 'ANACTRL')] [('alias', '', 1558794497092), ('catalog', 'C', 1558794497092), ('description', '模拟量控制', 1558794497092), ('name', 'ANACTRL', 1558794497092)]
[('id', 'ANALOG')] [('alias', '', 1558794497092), ('catalog', 'A', 1558794497092), ('description', '模拟量', 1558794497092), ('name', 'ANALOG', 1558794497092)]
[('id', 'BREAK')] [('alias', '', 1558794497092), ('catalog', 'D', 1558794497092), ('description', '开关状态', 1558794497092), ('name', 'BREAK', 1558794497092)]
[('id', 'BREAK_2')] [('alias', '', 1558794497092), ('catalog', 'D', 1558794497092), ('description', '双位开关状态', 1558794497092), ('name', 'BREAK_2', 1558794497092)]
[('id', 'BREAK_CAP')] [('alias', '', 1558794497092), ('catalog', 'D', 1558794497092), ('description', '电容器开关状态', 1558794497092), ('name', 'BREAK_CAP', 1558794497092)]
[('id', 'CAPACITOR')] [('alias',

TableName: master_info

PrimaryKey: [('id', 'STRING')]

[('id', 'svc_control')] [('master_pod', 'kemp-svc-control-0', 1566444737721), ('up_time', 1566444737701, 1566444737721)]
TableName: measure_type

PrimaryKey: [('id', 'STRING')]

[('id', 'Ag')] [('alias', 'Ag', 1558794496790), ('catalog', 'A', 1558794496790), ('description', '角度', 1558794496790), ('name', 'Ag', 1558794496790)]
[('id', 'Bv')] [('alias', 'Bv', 1558794496790), ('catalog', 'A', 1558794496790), ('description', '电池电压', 1558794496790), ('name', 'Bv', 1558794496790)]
[('id', 'Cb')] [('alias', 'Cb', 1558794496790), ('catalog', 'D', 1558794496790), ('description', '开关', 1558794496790), ('name', 'Cb', 1558794496790)]
[('id', 'Cf')] [('alias', 'Cf', 1558794496790), ('catalog', 'A', 1558794496790), ('description', '通道流量', 1558794496790), ('name', 'Cf', 1558794496790)]
[('id', 'Cs')] [('alias', 'Cs', 1558794496790), ('catalog', 'A', 1558794496790), ('description', '功率因数', 1558794496790), ('name', 'Cs', 1558794496790)]
[('id', 'D

[('id', '10000B13000007BAC000A0081000')] [('pt_index_id', '["DIG:201906:3"]', 1562237505502)]
[('id', '10000B13000007BAC002A0081000')] [('pt_index_id', '["DIG:201906:11"]', 1562237503937)]
[('id', '10000B13000007BAC003A0081000')] [('pt_index_id', '["DIG:201906:14"]', 1562237508161)]
[('id', '10000B13000007BAC005A0081000')] [('pt_index_id', '["DIG:201906:19"]', 1562237499658)]
[('id', '10000B13000007BAC006A0081000')] [('pt_index_id', '["DIG:201906:21"]', 1562237494213)]
[('id', '10000B13000007BAD001A0081000')] [('pt_index_id', '["DIG:201906:12"]', 1562237507940)]
[('id', '10000B13000007BAD002A0081000')] [('pt_index_id', '["DIG:201906:15"]', 1562237504661)]
[('id', '10000B13000007BAD004A0081000')] [('pt_index_id', '["DIG:201906:20"]', 1562237499007)]
[('id', '10000B13000007BAD005A0081000')] [('pt_index_id', '["DIG:201906:22"]', 1562237504881)]
[('id', '10000B13000007BAD022A0081000')] [('pt_index_id', '["DIG:201906:4"]', 1562237499587)]
TableName: point_value

PrimaryKey: [('id', 'STRING'

...

[('id', '50001')] [('d_deverrtimes', 1, 1568854205066), ('d_errtimes', 43, 1568964946673), ('d_mastertime', 9280, 1568964936699), ('d_runtime', 40600, 1568964936699), ('d_rxbytes', 7842543, 1568964902814), ('d_rxchkerrframes', 0, 1568822419307), ('d_rxerrframes', 0, 1568822419307), ('d_rxframes', 32116, 1568964902814), ('d_rxtimeoutframes', 1034, 1568886497643), ('d_txbytes', 30794, 1568964902814), ('d_txframes', 5074, 1568964902814), ('deaddataflag', 0, 1568885063754), ('deaddatatime', 1568885020, 1568885020149), ('deverrflag', 0, 1568856208702), ('deverrtime', 1568854205, 1568854205066), ('deverrtimes', 0, 1568854323006), ('down_time', 1568964937, 1568964946673), ('errtimes', 0, 1568965006821), ('init_time', 1568965933, 1568965935170), ('loserate', 0, 1568946697770), ('reason', '处理容器停止', 1568965935170), ('standby_time', 1568946839, 1568946839835), ('state', 'INIT', 1568965935170), ('up_time', 1568961410, 1568961412379)]
[('id', '50002')] [('d_deverrtimes', 1, 1568854205079), ('d_errt

[('id', '20001')] [('addr', 1, 1564539325172), ('alias', '', 1564539325172), ('alldatacallperiod', 10, 1564539325172), ('allkwhcallperiod', 10, 1564539325172), ('alm_mode', 0, 1564539325172), ('alm_priority', 0, 1564539325172), ('aop1', 0, 1565232925140), ('aop2', 0, 1564539325172), ('description', '', 1564539325172), ('firstacc', 0, 1564539325172), ('firstai', 0, 1564539325172), ('firstdi', 0, 1564539325172), ('firstdo', 0, 1564539325172), ('fwddatachangeval', 0.001, 1564539325172), ('iecmaxgroup', 0, 1564539325172), ('maxacc', 0, 1564539325172), ('maxai', 500, 1564539325172), ('maxdi', 200, 1564539325172), ('maxsoe', 1024, 1564539325172), ('name', 'test00001', 1564539325172), ('no', 1, 1564539325172), ('nodataswitchdelay', 60, 1564539325172), ('nodataswitchflag', True, 1564539325172), ('r_area_id', 'test_s1_1', 1564539325172), ('r_station_id', 'test00001', 1564539325172), ('rtu_type', 0, 1564539325172), ('subaddr', 1, 1564539325172), ('syntimeflag', True, 1564539325172), ('syntimeper

[('id', '005cefef6c9230729f9caed472954f57')] [('app_state', 'fertd', 1569373373581), ('app_state_time', 1569373365418, 1569373373581), ('d_run_time', 32593, 1569373394001), ('d_switchtimes', 1, 1569373383960), ('reason', '仅有一个通道运行', 1569373383960), ('route_state', '{"50453":"UP"}', 1569373383960), ('routeonduty', 'NONE', 1569373373581), ('state', 'UP', 1569373383960), ('up_time', 1569373376, 1569373383960)]
[('id', '00bce994ab8633da949b8deda95dfc8b')] [('app_state', 'fertd', 1569373394001), ('app_state_time', 1569373392922, 1569373394001), ('reason', '新的处理容器', 1569373394001), ('route_state', '{}', 1569373394001), ('routeonduty', 'NONE', 1569373394001), ('state', 'INIT', 1569373394001)]
[('id', '00cc6bfff8c13715a3275db51466752c')] [('app_state', 'fertd', 1569373963478), ('app_state_time', 1569373961740, 1569373963478), ('reason', '新的处理容器', 1569373963478), ('route_state', '{}', 1569373963478), ('routeonduty', 'NONE', 1569373963478), ('state', 'INIT', 1569373963478)]
[('id', '00e54d3a52a6

[('id', 'fert.group_channels')] [('alias', '', 1560163275263), ('config', '10', 1560163275263), ('description', '单个通道分组内通道最大数目', 1560163275263), ('name', 'fert.group_channels', 1560163275263)]
[('id', 'scada.twoyxdiff')] [('alias', 'scada.twoyxdiff', 1558794562015), ('config', '20', 1558794562015), ('description', 'scada.twoyxdiff', 1558794562015), ('name', 'scada.twoyxdiff', 1558794562015)]
TableName: system_version

PrimaryKey: [('id', 'STRING')]

TableName: tag_info

PrimaryKey: [('id', 'STRING')]

[('id', 'test00001.122:INHIBIT_ALARM')] [('comment', 'hhhhh', 1566385172121), ('node', '172.16.118.235', 1561714663037), ('obj_id', 'test00001.122', 1561714663037), ('tag_type', 'INHIBIT_ALARM', 1561714663037), ('user', '内置系统管理员', 1561714663037), ('x', 1, 1566385172121), ('y', 361, 1561714663037)]
[('id', 'test00001.51BC:INHIBIT_ALARM')] [('comment', '', 1560844436792), ('node', '218.56.32.4', 1560844436792), ('obj_id', 'test00001.51BC', 1560844436792), ('tag_type', 'INHIBIT_ALARM', 15608

[('no', 0)] [('time', 169, 1557559344803)]
[('no', 1)] [('time', 178, 1557559345251)]
[('no', 2)] [('time', 155, 1557559344899)]
[('no', 3)] [('time', 165, 1557559345125)]
[('no', 4)] [('time', 167, 1557559344459)]
TableName: volt_level

PrimaryKey: [('id', 'STRING')]

[('id', '10000B13000007/vol_10')] [('alias', '', 1562230453299), ('description', 'vol_10', 1562230453299), ('name', '10000B13000007/vol_10', 1562230453299), ('r_base_volt_id', '10kV', 1562230453299), ('r_station_id', '10000B13000007', 1562230453299)]
[('id', '10000B13000007/vol_110')] [('alias', '', 1562230453344), ('description', 'vol_110', 1562230453344), ('name', '10000B13000007/vol_110', 1562230453344), ('r_base_volt_id', '110kV', 1562230453344), ('r_station_id', '10000B13000007', 1562230453344)]
[('id', '10000B13000007/vol_35')] [('alias', '', 1562230453299), ('description', 'vol_35', 1562230453299), ('name', '10000B13000007/vol_35', 1562230453299), ('r_base_volt_id', '35kV', 1562230453299), ('r_station_id', '10000B

In [16]:
for table_name in table_names:
    for table, index_name in client.list_search_index(table_name):
        print(table, index_name)
        index_meta, sync_stat = client.describe_search_index(table, index_name)
        print(index_meta.__dict__)
        print(sync_stat.__dict__)

station station_dfe_index
{'fields': [<tablestore.metadata.FieldSchema object at 0x00000252CC1A5148>], 'index_setting': <tablestore.metadata.IndexSetting object at 0x00000252CC1A51C8>, 'index_sort': <tablestore.metadata.Sort object at 0x00000252CC1B0F48>}
{'sync_phase': <SyncPhase.INCR: 1>, 'current_sync_timestamp': 1576147109217000000}
